In [1]:
import pandas as pd
import os 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline

In [2]:
sepsis = pd.read_csv('Sepsis_Compiled_Full.csv',index_col=0)

/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
sepsis.head()

,PatientID,Delta_t_hours,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,Sepsis_At_All
0,p014977,0,80.0,100.0,36.50,121.00,58.0,41.00,13.5,NaN,1.0,25.0,1.0,7.40,47.0,77.0,NaN,53.0,NaN,NaN,111.0,2.1,NaN,107.0,2.4,NaN,NaN,3.7,NaN,NaN,27.9,8.8,38.1,9.9,223.0,160.0,77.27,1,0.0,1.0,-69.14,3,0,0
1,p014977,1,76.0,100.0,36.25,113.25,61.0,41.50,12.0,NaN,1.0,25.0,0.5,7.40,NaN,NaN,NaN,53.0,NaN,NaN,111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.9,8.8,NaN,9.9,NaN,NaN,77.27,1,0.0,1.0,-69.14,4,0,0
2,p014977,2,80.0,100.0,36.25,132.75,71.5,46.25,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.27,1,0.0,1.0,-69.14,5,0,0
3,p014977,3,78.0,100.0,36.10,103.50,58.0,43.00,12.0,NaN,-3.0,NaN,NaN,7.34,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.5,NaN,NaN,NaN,4.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.27,1,0.0,1.0,-69.14,6,0,0
4,p014977,4,74.0,100.0,36.00,128.75,69.5,44.50,12.5,NaN,-3.0,NaN,NaN,7.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.27,1,0.0,1.0,-69.14,7,0,0


In [16]:
test = sepsis[sepsis['PatientID'] == 'p014977']

### Input and output creation
* From the previous notebook, we found that we can capture 98% of the all patients if we use a cutoff of 100 for each patient
* Get all patients with intervals <= 100, T_max =100
* Add NaN's to the ends up 100
* Interpolate linearly for mid points, backfill for starting, and forward fill
* Use the Sepsis at All label (experiment late with the actuall sepsis label)
* input dimensions (num_samples,100, 44ish) 
* Experiment later with variable input


In [10]:
#aux Functions, make into class?
def check_frame_is_right_shape(frame,length=100):
    if frame.shape[0] <= length:
        return True
    else:
        return False
       
def pad_frame_to_length(frame,length=100):
    #subset frame
    frame = frame.iloc[:,list(range(2,37))+[-1]]
    #get size
    rows,cols = frame.shape[0],frame.shape[1]
    #broadcast last_row
    last_row = frame.iloc[-1,:]
    for i in range(length-rows):
        frame = frame.append(last_row,ignore_index=True)
    return frame

def fill_nans(frame,method='linear',length=100):
    #first interpolat linearly
    frame = frame.interpolate(method=method)
    #then backfill,forwardfill, then 0
    frame = frame.fillna(method='bfill').fillna(method='ffill').fillna(0)
    #assert nans
    assert np.sum(pd.isnull(frame.values)) == 0, 'There are still NaNs!'
    assert frame.shape[0] == length,'Not the right size!'
    return frame

def pull_x_y(frame):
    x = frame.iloc[:,:-1].values
    y = frame.iloc[:,-1].values
    return(x,y)    

In [19]:
x,y = pull_x_y(fill_nans(pad_frame_to_length(test)))


array([[ 80.  , 100.  ,  36.5 , ..., 223.  , 160.  ,  77.27],
       [ 76.  , 100.  ,  36.25, ..., 223.  , 161.8 ,  77.27],
       [ 80.  , 100.  ,  36.25, ..., 223.  , 163.6 ,  77.27],
       ...,
       [ 94.  ,  97.  ,  37.2 , ..., 223.  , 105.  ,  77.27],
       [ 94.  ,  97.  ,  37.2 , ..., 223.  , 105.  ,  77.27],
       [ 94.  ,  97.  ,  37.2 , ..., 223.  , 105.  ,  77.27]])

In [40]:
X_s, y_s = np.zeros((39680,100,35)),np.zeros((39680,100))
i = 0
start_time = time.time()
for a,b in sepsis.groupby('PatientID'):
    if i % 500 == 0:
        current_time = time.time()
        print(i,' th pass ',current_time - start_time)
    if check_frame_is_right_shape(b):
        frame = pad_frame_to_length(b)
        frame = fill_nans(frame)
        x,y = pull_x_y(frame)
        X_s[i,:,:] = x
        y_s[i,:] = y
    i += 1

0  th pass  0.6236889362335205
500  th pass  27.08515191078186
1000  th pass  56.055757999420166
1500  th pass  81.08309507369995
2000  th pass  106.28260707855225
2500  th pass  131.03501892089844
3000  th pass  156.6987099647522
3500  th pass  182.83502197265625
4000  th pass  209.28379702568054
4500  th pass  233.69968700408936
5000  th pass  260.6583318710327
5500  th pass  286.3210229873657
6000  th pass  310.35958194732666
6500  th pass  334.2575521469116
7000  th pass  358.54151606559753
7500  th pass  382.1903290748596
8000  th pass  406.25332617759705
8500  th pass  430.2260730266571
9000  th pass  454.3965570926666
9500  th pass  477.9534170627594
10000  th pass  502.0519471168518
10500  th pass  526.348240852356
11000  th pass  550.7445220947266
11500  th pass  577.1141929626465
12000  th pass  602.7730429172516
12500  th pass  628.2008309364319
13000  th pass  653.3876490592957
13500  th pass  678.7300539016724
14000  th pass  704.8960378170013
14500  th pass  731.771508932

IndexError: index 39680 is out of bounds for axis 0 with size 39680

In [46]:
X_s.shape,y_s.shape

((39680, 100, 35), (39680, 100))

In [47]:
#save files
np.save('Xs_window_100_v2.npy',X_s)
np.save('Ys_window_100_v2.npy',y_s)

In [57]:
test = sepsis[sepsis['PatientID'] == 'p018469']
check_frame_is_right_shape(test)

False

In [70]:
40000 // 500

80

In [78]:
1 - np.nan

nan